# Agents to research and write an article

---

## Setup

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## LLM

In [15]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt40125",
    temperature=0,
)

## Creating Agents

In [16]:
from crewai import Agent, Task, Crew

In [17]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
	verbose=True
)

In [18]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [19]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

In [20]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [21]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [22]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

In [23]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],  # order matters, will be executed sequentially
    verbose=2
)

2024-05-19 19:12:21,689 - 24664 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

In [24]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...


I now can give a great answer.

Final Answer:

**Content Plan Document for Blog Article on Artificial Intelligence**

**Title:** Navigating the Future: The Evolution and Impact of Artificial Intelligence

**Target Audience Analysis:**
- **Primary Audience:** Tech enthusiasts, AI researchers, and professionals in the technology sector who are keen on staying updated with the latest AI trends and innovations.
- **Secondary Audience:** Business owners, marketers, and decision-makers interested in leveraging AI for business growth and efficiency.
- **Interests:** Cutting-edge technology, AI applications in various industries, ethical considerations in AI, and AI's impact on the job market.
- **Pain Points:** Understanding complex AI concepts, integrating AI into existing systems, ethical concerns, and keeping up with rapid advancements.

**SEO Keywords:** Artificial Intelligence trends, AI innovations, AI in business, ethical AI, AI technology updates, AI impact on jobs.

**Content Outline

In [25]:
from IPython.display import Markdown

Markdown(result)

```markdown
# Navigating the Future: The Evolution and Impact of Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has evolved from the realms of science fiction to become an integral part of our daily lives and work. This evolution is not just a testament to human ingenuity but also a guiding light towards future possibilities. In this article, we aim to dissect the latest trends, spotlight key players, and delve into AI's broader impact, offering insights for tech enthusiasts, AI researchers, and professionals in the technology sector.

## The Latest Trends in AI
### Generative AI
The emergence of generative models marks a significant leap in AI's capabilities, enabling machines to create content that is increasingly indistinguishable from that produced by humans. From designing virtual fashion to generating new drug formulas, generative AI is pushing the boundaries of creativity and innovation.

### AI in Healthcare
AI's role in healthcare is becoming more pivotal, offering breakthroughs in diagnostics, personalized medicine, and patient care. These innovations not only promise to enhance the quality of healthcare services but also make them more accessible to underserved populations.

### Sustainable AI
In the face of climate change, AI offers a glimmer of hope. By optimizing energy consumption in industries and enabling smarter resource management, AI is at the forefront of promoting sustainability and combating environmental challenges.

### AI Ethics and Governance
As AI becomes more pervasive, ethical considerations and governance have sparked intense debate. Developing ethical AI and establishing governance frameworks are crucial to ensure that AI benefits humanity while minimizing harm.

## Key Players Shaping the Future of AI
### Tech Giants
Companies like Google, Amazon, and Microsoft are not just driving AI innovations; they're also setting the standards for its application across industries. Their contributions are pivotal in shaping the AI landscape.

### Startups to Watch
Innovative startups are challenging the status quo, introducing fresh perspectives and groundbreaking technologies in the AI field. These entities are crucial for fostering competition and innovation.

### Academic and Research Institutions
Leading universities and research labs are the unsung heroes of AI advancements. Their relentless pursuit of knowledge and innovation continues to push the boundaries of what AI can achieve.

## Noteworthy News in AI
Recent breakthroughs and policy changes are reshaping the AI landscape. From significant investments in AI startups to the introduction of AI governance policies, these developments have far-reaching implications for professionals and businesses in the tech industry.

## AI's Impact on Society and the Job Market
AI is transforming industries, creating new job opportunities, and simultaneously raising concerns about job displacement. Understanding the skills needed to thrive in an AI-driven future is essential for professionals looking to stay ahead in their careers.

## Navigating Ethical Considerations
The ethical dilemmas posed by AI, including issues of privacy, bias, and autonomy, are complex. Ongoing efforts to create ethical AI guidelines and frameworks are crucial in navigating these challenges responsibly.

## Conclusion and Call to Action
The transformative potential of AI is undeniable. As we continue to explore this evolving landscape, it's imperative that we engage with AI responsibly, staying informed about the latest trends and ethical considerations. We invite our readers to join the conversation, share their insights, and contribute to shaping the future of AI.

*Let's navigate the future of AI together, responsibly and with an eye towards the betterment of society.*
```

This revised blog post maintains the original's engaging structure while ensuring grammatical precision and alignment with the brand's voice. Each section is crafted to inform, engage, and provoke thought, encouraging readers to delve deeper into the fascinating world of artificial intelligence. The content is SEO-optimized, ensuring visibility while preserving the integrity of the information provided.